In [13]:
import numpy as np
part1 = np.load("part1a.npz")

In [14]:
part1.files
Ic_0 = part1["Ic_0"]
N = part1["N"]
gamma = part1["gamma"]
Lc = part1["Lc"]
Svc_0 = part1["Svc_0_pmf"]

In [15]:
S_0 = N-(Ic_0.sum())
I_0 = Ic_0.sum()
R_0 = 0

In [16]:
import random 
from random import Random
random.seed(a=None, version=2)
random.uniform(0, 0.25)

0.038521180175811076

In [17]:
import pandas as pd
data = pd.DataFrame(Svc_0)
data["Ic_0"] = Ic_0
data

,0,1,2,3,Ic_0
0,0.109533,0.033008,0.081790,0.115940,47
1,0.030898,0.085120,0.045288,0.057183,90
2,0.074715,0.024956,0.102366,0.042567,51
3,0.016059,0.081096,0.091582,0.007900,65


In [18]:
B0c = []
B1c = []
B2c = []
B3c = []
for i in range(4):
    B0c.append(random.uniform(0, 0.25))
    B1c.append(random.uniform(0.25, 0.5))
    B2c.append(random.uniform(0.5, 0.75))
    B3c.append(random.uniform(0.75, 1.0))

In [19]:
#B1c.sort()
#B2c.sort()
#B0c.sort()
#B3c.sort()
Bvc = np.array([B0c, B1c, B2c, B3c])
for i in range(4):
    Bvc[i].sort()

In [20]:
Bvc = pd.DataFrame(Bvc)
Bvc[0]
Bvc

,0,1,2,3
0,0.130711,0.131156,0.180407,0.241442
1,0.258765,0.383862,0.399211,0.484767
2,0.521315,0.579607,0.711366,0.723019
3,0.855882,0.885537,0.936600,0.988131


In [21]:
def S_rate(Svc, I, N, Bvc, gamma):
    """Calculate the rate of change of suceptible group split by vulnerability and comorbidity, return a
    tuple where the first element is split on both categories, and the second element only split by comorbidity"""
    #create empty data frame to store results
    dSvcdt = pd.DataFrame()
    #nested for loop, first level iterates through vulnerability
    forloop_start = time.time_ns()
    for v in range(4):
        #create an empty list for each comorbidity level
        dScdt = []
        for c in range(4):
            #here we calculate the value of dSdt for each comorbidity group in this iteration
            dScdt.append(-(Bvc[v][c] * (Svc[v][c]) * I) / N)
        #assign the list as a column in the data frame, at index c.
        dSvcdt[v] = dScdt
    forloop_end = time.time_ns()
    print("forloop", forloop_end - forloop_start/1000)
    sumstart = time.time_ns()
    dSdt = dSvcdt.sum(axis = 0)
    sumend = time.time_ns()
    print("sum", sumend - sumstart/1000)
    return dSvcdt, dSdt

forloop 1.6012966237578872e+18
sum 1.60129662375858e+18


0   -18.976661
1   -31.190348
2   -48.431876
3   -23.856897
dtype: float64

In [22]:
def I_rate(dSdt, Ic, gamma):
    """This function takes in the rate of change of S split by comorbidity, I split by comorbidity and gamma.
    It returns a tuple, the first element is dIdt split by comorbidity, the second element is the aggregate"""
    dIcdt = []
    dIdt = 0
    for c in range(4):
        #now we implement the rest of the formula
        #print(-(dSdt.iloc[c].sum()) - gamma*Ic[c])
        dIcdt.append(-dSdt[c] - gamma*Ic[c])
        dIdt += -dSdt[c] - gamma*Ic[c]
    #print(type(dIdt))
    return dIcdt, dIdt
def R_rate(dIdt, gamma):
    dRdt = gamma * dIdt[1]
    return dRdt

forloop 1.6012966246431273e+18
sum 1.6012966246440036e+18


([-3.3567260955079052,
  -6.42828398700941,
  -3.642512734506155,
  -4.642560597025817],
 -18.070083414049286)

In [25]:
import time
Svc = pd.DataFrame(Svc_0*N)
Ic = Ic_0
I = I_0
S = N - I
R = 0
simulation = [[S, I, R]]
for i in range(20):
    dSdt_start = time.time_ns()
    dSdt = S_rate(Svc, Ic, N, Bvc, gamma)
    #print(dSdt)
    dSdt_end = time.time_ns()
    print((dSdt_end-dSdt_start)/1000)
    dIdt = I_rate(dSdt[1], Ic, gamma)
    #print(dIdt)
    dRdt_start = time.time_ns()
    print((dSdt_end - dRdt_start)/1000)
    dRdt = R_rate(dIdt[1], gamma)
    #print(dRdt)
    dRdt_end = time.time_ns()
    print((dRdt_end - dRdt_start)/1000)
    Svc = Svc + dSdt[0]
    S = S + dSdt[1].sum()
    #print(S)
    Ic = Ic + dIdt[0]
    I = I + dIdt[1].sum()
    #print(I)
    R = R + dRdt
    simulation.append([[S, I, R]])
simulation

forloop 1.6012966291448156e+18
sum 1.601296629146383e+18
6072.0
-280.0
29.0
forloop 1.6012966291531005e+18
sum 1.601296629154179e+18
6216.0
-236.0
27.0
forloop 1.6012966291591332e+18
sum 1.6012966291609044e+18
5439.0
-528.0
47.0
forloop 1.6012966291663227e+18
sum 1.6012966291674798e+18
4127.0
-224.0
22.0
forloop 1.6012966291728934e+18
sum 1.6012966291739364e+18
4440.0
-313.0
21.0
forloop 1.6012966291791311e+18
sum 1.601296629180197e+18
4381.0
-667.0
30.0
forloop 1.6012966291876273e+18
sum 1.6012966291892186e+18
6587.0
-1003.0
138.0
forloop 1.6012966292074058e+18
sum 1.6012966292105859e+18
17794.0
-2580.0
241.0
forloop 1.6012966292546243e+18
sum 1.6012966292749176e+18
54623.0
-11936.0
216.0
forloop 1.601296629456801e+18
sum 1.6012966294961393e+18
182261.0
-47109.0
388.0
forloop 1.6012966302483535e+18
sum 1.6012966304544732e+18
824225.0
-248917.0
1628.0
forloop 1.601296633514265e+18
sum 1.6012966343823839e+18
3332385.0
-1028163.0
6409.0
forloop 1.601296661096903e+18
sum 1.601296698860118

KeyboardInterrupt: 

In [19]:
simulation = pd.DataFrame(np.array([[S_0, I, R]]), columns = ["S", "I", "R"])

In [37]:
simulation.append(["1", "2","3"])
simulation

AttributeError: 'numpy.ndarray' object has no attribute 'append'